# ORGANIZANDO OS DADOS DA MALHA VIÁRIA DE VIÇOSA - COM O ANEL

### IMPORTANDO O PACOTE psycopg2 QUE CONECTA O PYTHON COM O POSTGRE-SQL

In [50]:
import psycopg2 as pg
#import pandas as pd

### CONECTANDO AO BANCO DE DADOS

In [51]:
con = pg.connect(host='localhost', 
                database='dissertacao_artigo1',
                user='postgres', 
                password='admin')

cur = con.cursor() #CRIANDO UMA INSTÂNCIA PARA EXECUTAR COMANDOS EM SQL

# OBS: O servidor hospedado na máquina local será conectado no banco de dados nomeado DISSERTAÇÃO, que possui usuário postgres e senha admin.

### VENDO QUANTOS GRAFOS A MALHA POSSUI

In [52]:
tabela_grafos = 'vias_grafos_anel' #TABELA COM OS GRAFOS
sql = f'select max(id) from {tabela_grafos}' #COMANDO EM SQL A SER EXECUTADO
cur.execute(sql) #EXECUTANDO O COMANDO CRIADO
dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
id_max = dados_consultados[0][0] #ID MÁXIMO DA MALHA

### ATRIBUINDO O ID DO GRAFO SEM ANEL NO SHP COM ANEL VIARIO

In [4]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos_anel = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA COM O ANEL
tabela_grafos = 'vias_grafos' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos_a = 'vga' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA COM O ANEL
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA

#CONSULTANDO OS DADOS DE CADA GRAFO DO MALHA COM ANEL VIARIO:

for id_i in range(1, id_max + 1):

    sql = f"SELECT {abr_grafos}.id FROM {tabela_grafos} {abr_grafos}, {tabela_grafos_anel} {abr_grafos_a} WHERE ST_Contains({abr_grafos}.geom, {abr_grafos_a}.geom) and {abr_grafos_a}.id = {id_i}" #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ID DA TABELA DOS DOS GRAFOS (QUE JÁ ESTÁ COMPLETA).

    cur.execute(sql) #EXECUTANDO O COMANDO

    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS

    #INSERINDO AS INFORMAÇÕES CONSULTADAS:

    if dados_consultados == []: #CONFERINDO SE A CONSULTA É VAZIA. CASO SEJA, PULA A ITERAÇÃO.
        print(f'ID grafo anel: {id_i} -> Passa!\n')
        continue        

    else:
        sql = f"UPDATE {tabela_grafos_anel} SET id_grafo_semanel = '{dados_consultados[0][0]}' WHERE id = {id_i}" #COMANDO EM SQL A SER EXECUTADO. SERÁ ADICIONADO OS ATRIBUTOS CONSULTADOS NA MALHA COM ANEL VIARIO.

        cur.execute(sql) #EXECUTANDO O COMANDO

        con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

        print(f'ID grafo anel: {id_i} -> ID grafo sem anel: {dados_consultados[0][0]}\n')


ID grafo anel: 1 -> Passa!

ID grafo anel: 2 -> ID grafo sem anel: 7

ID grafo anel: 3 -> ID grafo sem anel: 14

ID grafo anel: 4 -> ID grafo sem anel: 3

ID grafo anel: 5 -> ID grafo sem anel: 17

ID grafo anel: 6 -> ID grafo sem anel: 19

ID grafo anel: 7 -> Passa!

ID grafo anel: 8 -> ID grafo sem anel: 4

ID grafo anel: 9 -> ID grafo sem anel: 5

ID grafo anel: 10 -> ID grafo sem anel: 6

ID grafo anel: 11 -> ID grafo sem anel: 20

ID grafo anel: 12 -> ID grafo sem anel: 60

ID grafo anel: 13 -> ID grafo sem anel: 15

ID grafo anel: 14 -> ID grafo sem anel: 21

ID grafo anel: 15 -> ID grafo sem anel: 26

ID grafo anel: 16 -> ID grafo sem anel: 32

ID grafo anel: 17 -> ID grafo sem anel: 33

ID grafo anel: 18 -> ID grafo sem anel: 34

ID grafo anel: 19 -> ID grafo sem anel: 22

ID grafo anel: 20 -> ID grafo sem anel: 38

ID grafo anel: 21 -> ID grafo sem anel: 39

ID grafo anel: 22 -> ID grafo sem anel: 40

ID grafo anel: 23 -> ID grafo sem anel: 41

ID grafo anel: 24 -> ID grafo se

### ATRIBUINDO NOME DAS VIAS PARA O BANCO DE DADOS DOS GRAFOS

In [6]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'nome_rua' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_nomeruas' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vnr' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'nome_rua' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA
atributo_comp_grafo = 'comp_nome_rua' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS COMPRIMENTOS DOS NOMES DAS RUAS NA TABELA DO SHP DOS GRAFOS

#ADICIONANDO O NOME DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
    
    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    if dados_consultados == []:
        continue
        
    else:
        
        lista_dados = []
        
        for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
            if dado[0] not in lista_dados:
                lista_dados.append(dado[0])

        lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n

        for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
            if dado != lista_dados[len(lista_dados) - 1]:
                lista_dados_str = lista_dados_str + dado + '; '
            else:
                lista_dados_str = lista_dados_str + dado

        sql = f'select sum(st_length({abr_dados}.geom)) from {tabela_dados} {abr_dados}, {tabela_grafos} {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i} group by {abr_dados}.{atributo_dados_grafo}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O COMPRIMENTO DE VIA DOATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.

        cur.execute(sql) #EXECUTANDO O COMANDO

        dados_consultados = cur.fetchall() #RETORNANDO OS DADOS

        lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS

        for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
            if dado[0] not in lista_dados:
                lista_dados.append(dado[0])

        lista_comp_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS DE COMPRIMENTOS SERÃO TRANSFORMADAS EM UMA STRING DA SEGUINTE FORMA: COMP_1; COMP_2; COMP_3; ... ; COMP_n

        for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
            if dado != lista_dados[len(lista_dados) - 1]:
                lista_comp_str = lista_comp_str + str(dado) + '; '
            else:
                lista_comp_str = lista_comp_str + str(dado)

        #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:

        sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
        cur.execute(sql) #EXECUTANDO O COMANDO
        con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

        print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.

        #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DE COMP DE CADA ATRIBUTO:

        sql = f"update {tabela_grafos} set {atributo_comp_grafo}='{lista_comp_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
        cur.execute(sql) #EXECUTANDO O COMANDO
        con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

        print(f'ID: {id_i} {atributo_comp_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.


ID: 2 nome_rua OK!
ID: 2 comp_nome_rua OK!
ID: 3 nome_rua OK!
ID: 3 comp_nome_rua OK!
ID: 4 nome_rua OK!
ID: 4 comp_nome_rua OK!
ID: 5 nome_rua OK!
ID: 5 comp_nome_rua OK!
ID: 6 nome_rua OK!
ID: 6 comp_nome_rua OK!
ID: 8 nome_rua OK!
ID: 8 comp_nome_rua OK!
ID: 9 nome_rua OK!
ID: 9 comp_nome_rua OK!
ID: 10 nome_rua OK!
ID: 10 comp_nome_rua OK!
ID: 11 nome_rua OK!
ID: 11 comp_nome_rua OK!
ID: 12 nome_rua OK!
ID: 12 comp_nome_rua OK!
ID: 13 nome_rua OK!
ID: 13 comp_nome_rua OK!
ID: 14 nome_rua OK!
ID: 14 comp_nome_rua OK!
ID: 15 nome_rua OK!
ID: 15 comp_nome_rua OK!
ID: 16 nome_rua OK!
ID: 16 comp_nome_rua OK!
ID: 17 nome_rua OK!
ID: 17 comp_nome_rua OK!
ID: 18 nome_rua OK!
ID: 18 comp_nome_rua OK!
ID: 19 nome_rua OK!
ID: 19 comp_nome_rua OK!
ID: 20 nome_rua OK!
ID: 20 comp_nome_rua OK!
ID: 21 nome_rua OK!
ID: 21 comp_nome_rua OK!
ID: 22 nome_rua OK!
ID: 22 comp_nome_rua OK!
ID: 23 nome_rua OK!
ID: 23 comp_nome_rua OK!
ID: 24 nome_rua OK!
ID: 24 comp_nome_rua OK!
ID: 25 nome_rua OK!
ID: 

### CONFERINDO OS DADOS QUE TIVERAM INFORMAÇÕES ADICONADAS 

In [59]:
#ESSA ESTRUTURA DE REPETIÇÃO PARA CONFERIR OS COMPRIMENTOS DE ARCO E SE AS INFORMAÇÕES ADICIONADAS ANTERIORMENTE ESTÃO CONSISTENTES:

list_len = [] #CRIANDO UMA LISTA EM BRANCO PARA RECEBER OS VALORES DE TRUE OU FALSE REFERENTE SE A QUANTIDADE DE RUAS PRESENTE NA LINHA DO SHP DOS GRAFOS É IGUAL A QUANTIDADE DE COMPRIMENTOS DE RUA

list_comp = [] #CRIANDO UMA LISTA EM BRANCO PARA RECEBER OS VALORES DE TRUE OU FALSE REFERENTE SE A A SOMA DOS COMPRIMENTOS DE CADA RUA É IGUAL AO COMPRIMENTO TOTAL DA LINHA DO SHP DOS GRAFOS

for id_i in range(1, id_max + 1): #ESTRUTURA DE REPETIÇÃO PARA PERCORRER DO ARCO 1 AO ARCO DE ID MAXIMO
    
    sql = f'select {atributo_dados}, {atributo_comp_grafo}, comp_via from "{tabela_grafos}" where id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO AS INFORMAÇÕES DE COMPRIMENTO DE VIA, NOME DE RUAS E COMPRIMENTO DE CADA NOME DE RUA DO SHP DOS GRAFOS.

    cur.execute(sql) #EXECUTANDO O COMANDO

    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    dados_nome = dados_consultados[0][0].split(';') #LISTA COM NOME DAS RUAS DO ARCO
    
    comp_dado = dados_consultados[0][1].split(';') #LISTA COM O COMPRIMENTO DE CADA RUA DA LISTA
    
    comp_total = round(dados_consultados[0][2], 4) #COMPRIMENTO TOTAL DO ARCO
    
    sum_comp = 0 #CRIANDO VARIAVEL PARA RECEBER A SOMA DE CADA COMPRIMENTO DE RUA
    
    for comp in comp_dado: #ADICIONANDO A VARIAVEL ANTERIOR A SOMA DOS COMPRIMENTOS DE RUA
        sum_comp = sum_comp + float(comp)
        
    sum_comp = round(sum_comp, 4) #ARRENDONDANDO PARA 4 CASAS DECIMAIS A SOMA DAS RUAS
    
    len_id = len(dados_nome) == len(comp_dado) #A QUANTIDADE DE RUAS É IGUAL A QUANTIDADE DO COMPRIMENTO DESSAS RUAS?
    
    list_len.append(len_id) #ADICIONANDO A VARIAVEL ANTERIOR A LISTA
    
    comp_id = sum_comp == comp_total #A SOMA DO COMPRIMENTO DAS RUAS É IGUAL AO COMPRIMENTO DO ARCO?
    
    list_comp.append(comp_id) #ADICIONANDO A VARIAVEL ANTERIOR A LISTA      
           
    print(f'===---===---===\nLinha de id {id_i}:\n\nTem a mesma quantidade de item em comp_ruas e nome_ruas?\n{len_id}\n\nComprimento total é igual ao somatorio de cada rua?\n{comp_id}\n\nSoma das ruas:{sum_comp}\nComprimento total do arco: {comp_total}\nDiferença: {comp_total - sum_comp}\n===---===---===\n') #MENSAGEM COM AS COMPARAÇÕES PARA CONFERIR A CONSISTENCIA ENTRE AS INFORMAÇÕES
   

===---===---===
Linha de id 1:

Tem a mesma quantidade de item em comp_ruas e nome_ruas?
True

Comprimento total é igual ao somatorio de cada rua?
True

Soma das ruas:6586.6376
Comprimento total do arco: 6586.6376
Diferença: 0.0
===---===---===

===---===---===
Linha de id 2:

Tem a mesma quantidade de item em comp_ruas e nome_ruas?
True

Comprimento total é igual ao somatorio de cada rua?
True

Soma das ruas:234.4648
Comprimento total do arco: 234.4648
Diferença: 0.0
===---===---===

===---===---===
Linha de id 3:

Tem a mesma quantidade de item em comp_ruas e nome_ruas?
True

Comprimento total é igual ao somatorio de cada rua?
True

Soma das ruas:467.812
Comprimento total do arco: 467.812
Diferença: 0.0
===---===---===

===---===---===
Linha de id 4:

Tem a mesma quantidade de item em comp_ruas e nome_ruas?
True

Comprimento total é igual ao somatorio de cada rua?
True

Soma das ruas:2755.4043
Comprimento total do arco: 2755.4043
Diferença: 0.0
===---===---===

===---===---===
Linha 

In [60]:
#QUANTIDADE DE FALSOS (QUANTIDADE DE ITENS)
list_len.count(False)

0

In [61]:
#QUANTIDADE DE FALTOS (SOMA DOS COMPRIMENTOS)
list_comp.count(False)

0

In [32]:
list_comp.index(False) + 1

ValueError: False is not in list

### ATRIBUINDO O TIPO DA VIA DE ACORDO COM O PLANO DE MOBILIDADE PARA O BANCO DE DADOS DOS GRAFOS

In [62]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'tipo_pm' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_tipopm' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vpm' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'tipo_pm' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA
atributo_comp_grafo = 'comp_tipo_pm' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS COMPRIMENTOS DOS NOMES DAS RUAS NA TABELA DO SHP DOS GRAFOS

#ADICIONANDO O TIPO DA VIA DE ACORDO COM O PLANO DE MOBILIDADE DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
    
    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i} group by {abr_dados}.{atributo_dados}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.

    cur.execute(sql) #EXECUTANDO O COMANDO

    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    if dados_consultados == []:
        continue
    
    else:
    
        lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS

        for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
            if dado[0] not in lista_dados:
                lista_dados.append(dado[0])

        lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n

        for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
            if dado != lista_dados[len(lista_dados) - 1]:
                lista_dados_str = lista_dados_str + dado + '; '
            else:
                lista_dados_str = lista_dados_str + dado

        sql = f'select sum(st_length({abr_dados}.geom)) from {tabela_dados} {abr_dados}, {tabela_grafos} {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i} group by {abr_dados}.{atributo_dados_grafo}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O COMPRIMENTO DE VIA DOATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.

        cur.execute(sql) #EXECUTANDO O COMANDO

        dados_consultados = cur.fetchall() #RETORNANDO OS DADOS

        lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS

        for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
            if dado[0] not in lista_dados:
                lista_dados.append(dado[0])

        lista_comp_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS DE COMPRIMENTOS SERÃO TRANSFORMADAS EM UMA STRING DA SEGUINTE FORMA: COMP_1; COMP_2; COMP_3; ... ; COMP_n

        for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
            if dado != lista_dados[len(lista_dados) - 1]:
                lista_comp_str = lista_comp_str + str(dado) + '; '
            else:
                lista_comp_str = lista_comp_str + str(dado)

        #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:

        sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
        cur.execute(sql) #EXECUTANDO O COMANDO
        con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

        print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.

        #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DE COMP DE CADA ATRIBUTO:

        sql = f"update {tabela_grafos} set {atributo_comp_grafo}='{lista_comp_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
        cur.execute(sql) #EXECUTANDO O COMANDO
        con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

        print(f'ID: {id_i} {atributo_comp_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.

ID: 2 tipo_pm OK!
ID: 2 comp_tipo_pm OK!
ID: 3 tipo_pm OK!
ID: 3 comp_tipo_pm OK!
ID: 4 tipo_pm OK!
ID: 4 comp_tipo_pm OK!
ID: 5 tipo_pm OK!
ID: 5 comp_tipo_pm OK!
ID: 6 tipo_pm OK!
ID: 6 comp_tipo_pm OK!
ID: 8 tipo_pm OK!
ID: 8 comp_tipo_pm OK!
ID: 9 tipo_pm OK!
ID: 9 comp_tipo_pm OK!
ID: 10 tipo_pm OK!
ID: 10 comp_tipo_pm OK!
ID: 11 tipo_pm OK!
ID: 11 comp_tipo_pm OK!
ID: 12 tipo_pm OK!
ID: 12 comp_tipo_pm OK!
ID: 13 tipo_pm OK!
ID: 13 comp_tipo_pm OK!
ID: 14 tipo_pm OK!
ID: 14 comp_tipo_pm OK!
ID: 15 tipo_pm OK!
ID: 15 comp_tipo_pm OK!
ID: 16 tipo_pm OK!
ID: 16 comp_tipo_pm OK!
ID: 17 tipo_pm OK!
ID: 17 comp_tipo_pm OK!
ID: 18 tipo_pm OK!
ID: 18 comp_tipo_pm OK!
ID: 19 tipo_pm OK!
ID: 19 comp_tipo_pm OK!
ID: 20 tipo_pm OK!
ID: 20 comp_tipo_pm OK!
ID: 21 tipo_pm OK!
ID: 21 comp_tipo_pm OK!
ID: 22 tipo_pm OK!
ID: 22 comp_tipo_pm OK!
ID: 23 tipo_pm OK!
ID: 23 comp_tipo_pm OK!
ID: 24 tipo_pm OK!
ID: 24 comp_tipo_pm OK!
ID: 25 tipo_pm OK!
ID: 25 comp_tipo_pm OK!
ID: 26 tipo_pm OK!
ID: 26

### CONFERINDO OS DADOS QUE TIVERAM INFORMAÇÕES ADICONADAS 

In [82]:
#ESSA ESTRUTURA DE REPETIÇÃO PARA CONFERIR OS COMPRIMENTOS DE ARCO E SE AS INFORMAÇÕES ADICIONADAS ANTERIORMENTE ESTÃO CONSISTENTES:

list_len = [] #CRIANDO UMA LISTA EM BRANCO PARA RECEBER OS VALORES DE TRUE OU FALSE REFERENTE SE A QUANTIDADE DE RUAS PRESENTE NA LINHA DO SHP DOS GRAFOS É IGUAL A QUANTIDADE DE COMPRIMENTOS DE RUA

list_comp = [] #CRIANDO UMA LISTA EM BRANCO PARA RECEBER OS VALORES DE TRUE OU FALSE REFERENTE SE A A SOMA DOS COMPRIMENTOS DE CADA RUA É IGUAL AO COMPRIMENTO TOTAL DA LINHA DO SHP DOS GRAFOS

for id_i in range(1, id_max + 1): #ESTRUTURA DE REPETIÇÃO PARA PERCORRER DO ARCO 1 AO ARCO DE ID MAXIMO
    
    sql = f'select {atributo_dados}, {atributo_comp_grafo}, comp_via from "{tabela_grafos}" where id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO AS INFORMAÇÕES DE COMPRIMENTO DE VIA, NOME DE RUAS E COMPRIMENTO DE CADA NOME DE RUA DO SHP DOS GRAFOS.

    cur.execute(sql) #EXECUTANDO O COMANDO

    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    dados_nome = dados_consultados[0][0].split(';') #LISTA COM NOME DAS RUAS DO ARCO
    
    comp_dado = dados_consultados[0][1].split(';') #LISTA COM O COMPRIMENTO DE CADA RUA DA LISTA
    
    comp_total = round(dados_consultados[0][2], 4) #COMPRIMENTO TOTAL DO ARCO
    
    sum_comp = 0 #CRIANDO VARIAVEL PARA RECEBER A SOMA DE CADA COMPRIMENTO DE RUA
    
    for comp in comp_dado: #ADICIONANDO A VARIAVEL ANTERIOR A SOMA DOS COMPRIMENTOS DE RUA
        sum_comp = sum_comp + float(comp)
        
    sum_comp = round(sum_comp, 4) #ARRENDONDANDO PARA 4 CASAS DECIMAIS A SOMA DAS RUAS
    
    len_id = len(dados_nome) == len(comp_dado) #A QUANTIDADE DE RUAS É IGUAL A QUANTIDADE DO COMPRIMENTO DESSAS RUAS?
    
    list_len.append(len_id) #ADICIONANDO A VARIAVEL ANTERIOR A LISTA
    
    comp_id = sum_comp == comp_total #A SOMA DO COMPRIMENTO DAS RUAS É IGUAL AO COMPRIMENTO DO ARCO?
    
    list_comp.append(comp_id) #ADICIONANDO A VARIAVEL ANTERIOR A LISTA      
           
    print(f'===---===---===\nLinha de id {id_i}:\n\nTem a mesma quantidade de item em comp_tipo_pm e tipo_pm?\n{len_id}\n\nComprimento total é igual ao somatorio de cada rua?\n{comp_id}\n\nSoma das ruas:{sum_comp}\nComprimento total do arco: {comp_total}\nDiferença: {comp_total - sum_comp}\n===---===---===\n') #MENSAGEM COM AS COMPARAÇÕES PARA CONFERIR A CONSISTENCIA ENTRE AS INFORMAÇÕES
   

===---===---===
Linha de id 1:

Tem a mesma quantidade de item em comp_tipo_pm e tipo_pm?
True

Comprimento total é igual ao somatorio de cada rua?
True

Soma das ruas:6586.6376
Comprimento total do arco: 6586.6376
Diferença: 0.0
===---===---===

===---===---===
Linha de id 2:

Tem a mesma quantidade de item em comp_tipo_pm e tipo_pm?
True

Comprimento total é igual ao somatorio de cada rua?
True

Soma das ruas:234.4648
Comprimento total do arco: 234.4648
Diferença: 0.0
===---===---===

===---===---===
Linha de id 3:

Tem a mesma quantidade de item em comp_tipo_pm e tipo_pm?
True

Comprimento total é igual ao somatorio de cada rua?
True

Soma das ruas:467.812
Comprimento total do arco: 467.812
Diferença: 0.0
===---===---===

===---===---===
Linha de id 4:

Tem a mesma quantidade de item em comp_tipo_pm e tipo_pm?
True

Comprimento total é igual ao somatorio de cada rua?
True

Soma das ruas:2755.4043
Comprimento total do arco: 2755.4043
Diferença: 0.0
===---===---===

===---===---===
Li

In [83]:
#QUANTIDADE DE FALSOS (QUANTIDADE DE ITENS)
list_len.count(False)

0

In [84]:
list_len.index(False) + 1

ValueError: False is not in list

In [85]:
#QUANTIDADE DE FALTOS (SOMA DOS COMPRIMENTOS)
list_comp.count(False)

0

In [86]:
list_comp.index(False) + 1

ValueError: False is not in list

### ATRIBUINDO O SENTIDO DA VIA PARA O BANCO DE DADOS DOS GRAFOS

In [48]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos_anel' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'oneway' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_oneway' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vow' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'oneway' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO SENTIDO DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
    
    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
    
    for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
        if dado[0] not in lista_dados:
            lista_dados.append(dado[0])
    
    lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n
    
    for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
        if dado != lista_dados[len(lista_dados) - 1]:
            lista_dados_str = lista_dados_str + dado + '; '
        else:
            lista_dados_str = lista_dados_str + dado
    
    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:
    
    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO
    
    print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.


ID: 1 oneway OK!
ID: 2 oneway OK!
ID: 3 oneway OK!
ID: 4 oneway OK!
ID: 5 oneway OK!
ID: 6 oneway OK!
ID: 7 oneway OK!
ID: 8 oneway OK!
ID: 9 oneway OK!
ID: 10 oneway OK!
ID: 11 oneway OK!
ID: 12 oneway OK!
ID: 13 oneway OK!
ID: 14 oneway OK!
ID: 15 oneway OK!
ID: 16 oneway OK!
ID: 17 oneway OK!
ID: 18 oneway OK!
ID: 19 oneway OK!
ID: 20 oneway OK!
ID: 21 oneway OK!
ID: 22 oneway OK!
ID: 23 oneway OK!
ID: 24 oneway OK!
ID: 25 oneway OK!
ID: 26 oneway OK!
ID: 27 oneway OK!
ID: 28 oneway OK!
ID: 29 oneway OK!
ID: 30 oneway OK!
ID: 31 oneway OK!
ID: 32 oneway OK!
ID: 33 oneway OK!
ID: 34 oneway OK!
ID: 35 oneway OK!
ID: 36 oneway OK!
ID: 37 oneway OK!
ID: 38 oneway OK!
ID: 39 oneway OK!
ID: 40 oneway OK!
ID: 41 oneway OK!
ID: 42 oneway OK!
ID: 43 oneway OK!
ID: 44 oneway OK!
ID: 45 oneway OK!
ID: 46 oneway OK!
ID: 47 oneway OK!
ID: 48 oneway OK!
ID: 49 oneway OK!
ID: 50 oneway OK!
ID: 51 oneway OK!
ID: 52 oneway OK!
ID: 53 oneway OK!
ID: 54 oneway OK!
ID: 55 oneway OK!
ID: 56 oneway OK!
I

In [87]:
cur.close()
con.close()